# Import Libaray

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import json

In [2]:
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [3]:
# Convert all text into lowercase form
df['sentence'] = df['sentence'].str.lower()
 
# Removing stopwords
stop_word = set(stopwords.words('english'))
 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Split the dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Tokenizing
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter to remove symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Save the word_index into a json file
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Creating sequences and padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

In [4]:
# Modeling
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
 
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

Epoch 1/30
25/25 [==============================] - 1s 9ms/step - loss: 0.6885 - accuracy: 0.5650 - val_loss: 0.7318 - val_accuracy: 0.2400
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.5650 - val_loss: 0.7686 - val_accuracy: 0.2400
Epoch 3/30
25/25 [==============================] - 0s 2ms/step - loss: 0.6758 - accuracy: 0.5650 - val_loss: 0.7767 - val_accuracy: 0.2400
Epoch 4/30
25/25 [==============================] - 0s 2ms/step - loss: 0.6615 - accuracy: 0.5725 - val_loss: 0.7691 - val_accuracy: 0.2450
Epoch 5/30
25/25 [==============================] - 0s 2ms/step - loss: 0.6214 - accuracy: 0.6600 - val_loss: 0.7641 - val_accuracy: 0.2800
Epoch 6/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5296 - accuracy: 0.7875 - val_loss: 0.7115 - val_accuracy: 0.4550
Epoch 7/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3832 - accuracy: 0.8938 - val_loss: 0.6155 - val_accuracy: 0.6500
Epoch 8/30
25/25 [==

In [5]:
# saved model
model.save("model.h5")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
